# 0-1 Knapsack example

Goal: Maximize the value of items in a bag constrained by the maximum weight the bag can carry.  At most one of each item can go in the bag.

## Setup

In [1]:
# General imports
import numpy as np

# Qiskit ansatz circuits
from qiskit.circuit.library import TwoLocal

# Qiskit primitives
from qiskit.primitives import Estimator as QiskitEstimator
from qiskit.primitives import Sampler as QiskitSampler

# Qiskit runtime
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Estimator, Sampler, Session

# optimization passes
from quadratic_program import QuadraticProgram
from workflows import QuadraticConverter, QuadraticPostprocess

# translations
from translators import docplex_mp_to_qp
from translators import qubo_to_sparse_pauli_op

# Docplex
from docplex.mp.model import Model

# SPSA
from spsa import minimize_spsa

## Load the Runtime (if using)

In [2]:
#service = QiskitRuntimeService()

In [3]:
#backend = service.get_backend('ibm_nazca')

## **Step 1**: Problem specification using standard tools and conversion to Quantum native format

Specify optimization problem using docplex and convert to Quadratic Unconstained Binary Opimization (QUBO) problem that can be cast as an Ising Hamiltonian suitable for a quantum solution.

In [4]:
value_of_items = [4, 3, 5, 6, 7]
weight_of_items = [2, 2, 4, 5, 6]
max_weight = 12

In [5]:
mdl = Model(name="0-1 Knapsack problem")
x = {i: mdl.binary_var(name=f"x_{i}") for i in range(len(value_of_items))}
mdl.maximize(mdl.sum(value_of_items[i] * x[i] for i in x))
mdl.add_constraint(mdl.sum(weight_of_items[i] * x[i] for i in x) <= max_weight);
print(mdl.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: 0-1 Knapsack problem

Maximize
 obj: 4 x_0 + 3 x_1 + 5 x_2 + 6 x_3 + 7 x_4
Subject To
 c1: 2 x_0 + 2 x_1 + 4 x_2 + 5 x_3 + 6 x_4 <= 12

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1

Binaries
 x_0 x_1 x_2 x_3 x_4
End



### Convert to `QuadraticProgram` format

In [6]:
qp = docplex_mp_to_qp(mdl)
print(qp.prettyprint())

Problem name: 0-1 Knapsack problem

Maximize
  4*x_0 + 3*x_1 + 5*x_2 + 6*x_3 + 7*x_4

Subject to
  Linear constraints (1)
    2*x_0 + 2*x_1 + 4*x_2 + 5*x_3 + 6*x_4 <= 12  'c0'

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4



### Classical transformation to QUBO problem and Ising Hamiltonian

In [7]:
quadratic_transformer = QuadraticConverter()
qubo = quadratic_transformer.run(qp)
print(qubo.prettyprint())

Problem name: 0-1 Knapsack problem

Minimize
  26*c0@int_slack@0^2 + 104*c0@int_slack@0*c0@int_slack@1
  + 208*c0@int_slack@0*c0@int_slack@2 + 260*c0@int_slack@0*c0@int_slack@3
  + 104*c0@int_slack@1^2 + 416*c0@int_slack@1*c0@int_slack@2
  + 520*c0@int_slack@1*c0@int_slack@3 + 416*c0@int_slack@2^2
  + 1040*c0@int_slack@2*c0@int_slack@3 + 650*c0@int_slack@3^2
  + 104*x_0*c0@int_slack@0 + 208*x_0*c0@int_slack@1 + 416*x_0*c0@int_slack@2
  + 520*x_0*c0@int_slack@3 + 104*x_0^2 + 208*x_0*x_1 + 416*x_0*x_2 + 520*x_0*x_3
  + 624*x_0*x_4 + 104*x_1*c0@int_slack@0 + 208*x_1*c0@int_slack@1
  + 416*x_1*c0@int_slack@2 + 520*x_1*c0@int_slack@3 + 104*x_1^2 + 416*x_1*x_2
  + 520*x_1*x_3 + 624*x_1*x_4 + 208*x_2*c0@int_slack@0 + 416*x_2*c0@int_slack@1
  + 832*x_2*c0@int_slack@2 + 1040*x_2*c0@int_slack@3 + 416*x_2^2 + 1040*x_2*x_3
  + 1248*x_2*x_4 + 260*x_3*c0@int_slack@0 + 520*x_3*c0@int_slack@1
  + 1040*x_3*c0@int_slack@2 + 1300*x_3*c0@int_slack@3 + 650*x_3^2 + 1560*x_3*x_4
  + 312*x_4*c0@int_slack@0 + 

In [8]:
hamiltonian, offset = qubo_to_sparse_pauli_op(qubo)
hamiltonian

SparsePauliOp(['IIIIIIIIZ', 'IIIIIIIZI', 'IIIIIIZII', 'IIIIIZIII', 'IIIIZIIII', 'IIIZIIIII', 'IIZIIIIII', 'IZIIIIIII', 'ZIIIIIIII', 'IIIIIIIZZ', 'IIIIIIZIZ', 'IIIIIZIIZ', 'IIIIZIIIZ', 'IIIZIIIIZ', 'IIZIIIIIZ', 'IZIIIIIIZ', 'ZIIIIIIIZ', 'IIIIIIZZI', 'IIIIIZIZI', 'IIIIZIIZI', 'IIIZIIIZI', 'IIZIIIIZI', 'IZIIIIIZI', 'ZIIIIIIZI', 'IIIIIZZII', 'IIIIZIZII', 'IIIZIIZII', 'IIZIIIZII', 'IZIIIIZII', 'ZIIIIIZII', 'IIIIZZIII', 'IIIZIZIII', 'IIZIIZIII', 'IZIIIZIII', 'ZIIIIZIII', 'IIIZZIIII', 'IIZIZIIII', 'IZIIZIIII', 'ZIIIZIIII', 'IIZZIIIII', 'IZIZIIIII', 'ZIIZIIIII', 'IZZIIIIII', 'ZIZIIIIII', 'ZZIIIIIII'],
              coeffs=[-180. +0.j, -180.5+0.j, -361.5+0.j, -452. +0.j, -542.5+0.j,  -91. +0.j,
 -182. +0.j, -364. +0.j, -455. +0.j,   52. +0.j,  104. +0.j,  130. +0.j,
  156. +0.j,   26. +0.j,   52. +0.j,  104. +0.j,  130. +0.j,  104. +0.j,
  130. +0.j,  156. +0.j,   26. +0.j,   52. +0.j,  104. +0.j,  130. +0.j,
  260. +0.j,  312. +0.j,   52. +0.j,  104. +0.j,  208. +0.j,  260. +0.j,
  390. +0.j, 

## **Step 2**: Setup quantum solution method and optimize

### Select ansatz circuit from circuit library

In [9]:
ansatz = TwoLocal(hamiltonian.num_qubits, 'ry', 'cx', 'linear', reps=1)

## **Step 3**: Solve using quantum primitives

### Standard cost function definition

In [10]:
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    cost = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    return cost

### Setup estimator and sampler instances

In [11]:
estimator = QiskitEstimator(options={"shots": int(1e4)})
sampler = QiskitSampler(options={"shots": int(1e4)})

### Perform minimization

In [12]:
x0 = 2*np.pi*np.random.random(size=ansatz.num_parameters)
res = minimize_spsa(cost_func, x0,
                    args=(ansatz, hamiltonian, estimator),
                    maxiter=100)
res

 message: Optimization terminated successfully.
 success: True
     fun: -441.822082474664
       x: [-7.304e+02 -1.988e+03 ... -2.612e+03 -3.465e+03]
     nit: 100
    nfev: 200

### Computute distribution at found minimum

In [13]:
# Assign solution parameters to ansatz
qc = ansatz.assign_parameters(res.x)
qc.measure_all()
samp_dist = sampler.run(qc, shots=int(1e4)).result().quasi_dists[0]

## **Step 4**: Post-process and return answer in classical format
Transform quantum solution and convert back into classical variable space

In [14]:
solution = QuadraticPostprocess(qubo, quadratic_transformer).run(samp_dist)
solution

array([1., 0., 1., 0., 1.])

### Intepretation of solution

In [15]:
print('Solution value:', sum(np.array(value_of_items)*solution))
print('Solution weight:', sum(np.array(weight_of_items)*solution))

Solution value: 16.0
Solution weight: 12.0
